In [1]:
import pandas as pd
import matplotlib.pyplot as plt
%matplotlib inline
import numpy as np
from scipy.stats import gaussian_kde

from sklearn.model_selection import train_test_split

# Constants

In [2]:
alpha = -0.1

# Preprocessing

In [3]:
plt.rcParams["figure.figsize"] = (15, 10)
pd.options.display.max_columns = 100

In [20]:
%%time
df = pd.read_csv('data/train_2011_2012_2013.csv', sep=";", nrows=1e10)

Wall time: 2min 6s


In [21]:
df.head()

,DATE,DAY_OFF,DAY_DS,WEEK_END,DAY_WE_DS,TPER_TEAM,TPER_HOUR,SPLIT_COD,ACD_COD,ACD_LIB,ASS_SOC_MERE,ASS_DIRECTORSHIP,ASS_ASSIGNMENT,ASS_PARTNER,ASS_POLE,ASS_BEGIN,ASS_END,ASS_COMENT,CSPL_I_STAFFTIME,CSPL_I_AVAILTIME,CSPL_I_ACDTIME,CSPL_I_ACWTIME,CSPL_I_ACWOUTTIME,CSPL_I_ACWINTIME,CSPL_I_AUXOUTTIME,CSPL_I_AUXINTIME,CSPL_I_OTHERTIME,CSPL_ACWINCALLS,CSPL_ACWINTIME,CSPL_AUXINCALLS,CSPL_AUXINTIME,CSPL_ACWOUTCALLS,CSPL_ACWOUTIME,CSPL_ACWOUTOFFCALLS,CSPL_ACWOUTOFFTIME,CSPL_AUXOUTCALLS,CSPL_AUXOUTTIME,CSPL_AUXOUTOFFCALLS,CSPL_AUXOUTOFFTIME,CSPL_INFLOWCALLS,CSPL_ACDCALLS,CSPL_ANSTIME,CSPL_HOLDCALLS,CSPL_HOLDTIME,CSPL_HOLDABNCALLS,CSPL_TRANSFERED,CSPL_CONFERENCE,CSPL_ABNCALLS,CSPL_ABNTIME,CSPL_ABNCALLS1,CSPL_ABNCALLS2,CSPL_ABNCALLS3,CSPL_ABNCALLS4,CSPL_ABNCALLS5,CSPL_ABNCALLS6,CSPL_ABNCALLS7,CSPL_ABNCALLS8,CSPL_ABNCALLS9,CSPL_ABNCALLS10,CSPL_OUTFLOWCALLS,CSPL_OUTFLOWTIME,CSPL_MAXINQUEUE,CSPL_CALLSOFFERED,CSPL_I_RINGTIME,CSPL_RINGTIME,CSPL_RINGCALLS,CSPL_NOANSREDIR,CSPL_MAXSTAFFED,CSPL_ACWOUTADJCALLS,CSPL_AUXOUTADJCALLS,CSPL_DEQUECALLS,CSPL_DEQUETIME,CSPL_DISCCALLS,CSPL_DISCTIME,CSPL_INTRVL,CSPL_INCOMPLETE,CSPL_ACCEPTABLE,CSPL_SERVICELEVEL,CSPL_ACDAUXOUTCALLS,CSPL_SLVLABNS,CSPL_SLVLOUTFLOWS,CSPL_RECEIVED_CALLS,CSPL_ABANDONNED_CALLS,CSPL_CALLS,CSPL_ACWTIME,CSPL_ACDTIME
0,2011-04-24 01:30:00.000,0,NaN,1,Dimanche,Nuit,1,855,1,Entity1 G3RV6,Entity1 France,DSIG,Téléphonie,Téléphonie,SUPPORT,2001-01-01 00:00:00.000,9999-12-31 00:00:00.000,NaN,5400,4812,0,0,0,0,0,0,543,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,3,0,0,0,0,0,0,30,0,0,30,0,0,0,0,0,0,0,0
1,2011-04-24 01:30:00.000,0,NaN,1,Dimanche,Nuit,1,1587,1,Entity1 G3RV6,Entity1 France,DSIG,Téléphonie,Téléphonie,SUPPORT,2001-01-01 00:00:00.000,9999-12-31 00:00:00.000,NaN,12600,2632,0,0,0,0,0,0,498,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,7,0,0,0,0,0,0,30,0,0,15,0,0,0,0,0,0,0,0
2,2011-04-24 01:30:00.000,0,NaN,1,Dimanche,Nuit,1,1589,1,Entity1 G3RV6,Entity1 France,DSIG,Téléphonie,Téléphonie,SUPPORT,2001-01-01 00:00:00.000,9999-12-31 00:00:00.000,NaN,7200,2632,0,0,0,0,0,0,498,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,4,0,0,0,0,0,0,30,0,0,20,0,0,0,0,0,0,0,0
3,2011-04-24 01:30:00.000,0,NaN,1,Dimanche,Nuit,1,1591,1,Entity1 G3RV6,Entity1 France,DSIG,Téléphonie,Téléphonie,SUPPORT,2001-01-01 00:00:00.000,9999-12-31 00:00:00.000,NaN,10800,832,0,0,0,0,0,0,498,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,6,0,0,0,0,0,0,30,0,0,0,0,0,0,0,0,0,0,0
4,2011-04-24 01:30:00.000,0,NaN,1,Dimanche,Nuit,1,1555,1,Entity1 G3RV6,Entity1 France,DSIG,Téléphonie,Téléphonie,SUPPORT,2001-01-01 00:00:00.000,9999-12-31 00:00:00.000,NaN,12600,2632,488,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,1,10,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,1,1,10,10,1,0,7,0,0,0,0,0,0,30,0,1,15,0,0,0,1,0,1,0,488


In [22]:
target = "CSPL_RECEIVED_CALLS"

### deleting all columns but very few

In [23]:
df = df[['DATE', target, 'ASS_ASSIGNMENT']]

In [24]:
df.head()

,DATE,CSPL_RECEIVED_CALLS,ASS_ASSIGNMENT
0,2011-04-24 01:30:00.000,0,Téléphonie
1,2011-04-24 01:30:00.000,0,Téléphonie
2,2011-04-24 01:30:00.000,0,Téléphonie
3,2011-04-24 01:30:00.000,0,Téléphonie
4,2011-04-24 01:30:00.000,1,Téléphonie


In [50]:
df.shape

(10878470, 3)

In [54]:
df = pd.DataFrame({target : df.groupby(["DATE", "ASS_ASSIGNMENT"])[target].sum()}).reset_index()

In [55]:
df.head()

,DATE,ASS_ASSIGNMENT,CSPL_RECEIVED_CALLS
0,2011-01-01 00:00:00.000,Crises,0
1,2011-01-01 00:00:00.000,Domicile,0
2,2011-01-01 00:00:00.000,Gestion,0
3,2011-01-01 00:00:00.000,Gestion - Accueil Telephonique,0
4,2011-01-01 00:00:00.000,Gestion Amex,0


In [61]:
df.DATE = pd.to_datetime(df.DATE)

In [62]:
import datetime

get the time slot of the day (0 for 00:00, 1 for 00:30, ... 47 for 23:30)

In [63]:
def get_slot(timestamp):
    assert timestamp.minute in [0, 30]
    return 2 * timestamp.hour + (timestamp.minute == 30)

In [64]:
%%time
df['slot'] = df.DATE.apply(get_slot)

Wall time: 3.12 s


In [65]:
%%time
df['dayofweek'] = df.DATE.apply(lambda x: x.dayofweek)

Wall time: 3.16 s


In [66]:
%%time
df['ass_hashed'] = df.ASS_ASSIGNMENT.apply(hash)

Wall time: 245 ms


In [67]:
df.head()

,DATE,ASS_ASSIGNMENT,CSPL_RECEIVED_CALLS,slot,dayofweek,ass_hashed
0,2011-01-01,Crises,0,0,5,-272063095
1,2011-01-01,Domicile,0,0,5,-1965427828
2,2011-01-01,Gestion,0,0,5,822364246
3,2011-01-01,Gestion - Accueil Telephonique,0,0,5,2096569015
4,2011-01-01,Gestion Amex,0,0,5,481775818


In [69]:
X_df = df[['ass_hashed', 'slot', 'dayofweek']]
y_df = df[target]

# Training

In [70]:
from sklearn.model_selection import ShuffleSplit

skf = ShuffleSplit(n_splits=2, test_size=0.2, random_state=57)
skf_is = list(skf.split(X_df))[0]
train_is, test_is = skf_is

X_train_df = X_df.iloc[train_is].copy()                                  
y_train_df = y_df.iloc[train_is].copy()
X_test_df = X_df.iloc[test_is].copy()                                    
y_test_df = y_df.iloc[test_is].copy() 

In [270]:
from sklearn.ensemble import RandomForestRegressor

rf = RandomForestRegressor(max_leaf_nodes=100, n_estimators=10, n_jobs=-1, oob_score=True, random_state=42)

In [271]:
%%time
rf.fit(X_train_df, y_train_df)

Wall time: 3.4 s


RandomForestRegressor(bootstrap=True, criterion='mse', max_depth=None,
           max_features='auto', max_leaf_nodes=100,
           min_impurity_split=1e-07, min_samples_leaf=1,
           min_samples_split=2, min_weight_fraction_leaf=0.0,
           n_estimators=10, n_jobs=-1, oob_score=True, random_state=42,
           verbose=0, warm_start=False)

In [272]:
y_pred_df = rf.predict(X_test_df)

# Evaluation

In [273]:
def lin_ex_scalar(y, y_hat):
    return np.exp(alpha * (y - y_hat)) - alpha * (y - y_hat) - 1

In [274]:
def lin_ex(y_df, y_est_df):
    return lin_ex_scalar(y_df, y_est_df)

In [275]:
error = lin_ex(y_test_df, y_pred_df)

In [276]:
error.sum()

45746014759529.04

# Read submission data

In [277]:
sub = pd.read_csv('data/submission.txt', sep='\t')

In [278]:
sub.head()

,DATE,ASS_ASSIGNMENT,prediction
0,2012-12-28 00:00:00.000,CMS,0
1,2012-12-28 00:00:00.000,Crises,0
2,2012-12-28 00:00:00.000,Domicile,0
3,2012-12-28 00:00:00.000,Gestion,0
4,2012-12-28 00:00:00.000,Gestion - Accueil Telephonique,0


In [279]:
sub.DATE = pd.to_datetime(sub.DATE)

In [280]:
sub.head()

,DATE,ASS_ASSIGNMENT,prediction
0,2012-12-28,CMS,0
1,2012-12-28,Crises,0
2,2012-12-28,Domicile,0
3,2012-12-28,Gestion,0
4,2012-12-28,Gestion - Accueil Telephonique,0


In [281]:
# there are no training data for the submission dates!
sub_ix = sub.set_index('DATE').index
df_ix = df.set_index("DATE").index
df_ix.intersection(sub_ix)

DatetimeIndex([], dtype='datetime64[ns]', name=u'DATE', freq=None)

In [282]:
sub.shape

(82909, 3)

In [283]:
sub['dayofweek'] = sub.DATE.apply(lambda x: x.dayofweek)
sub['slot'] = sub.DATE.apply(get_slot)

In [284]:
sub['ass_hashed'] = sub.ASS_ASSIGNMENT.apply(hash)

In [285]:
sub.head()

,DATE,ASS_ASSIGNMENT,prediction,dayofweek,slot,ass_hashed
0,2012-12-28,CMS,0,4,0,-1466160996
1,2012-12-28,Crises,0,4,0,-272063095
2,2012-12-28,Domicile,0,4,0,-1965427828
3,2012-12-28,Gestion,0,4,0,822364246
4,2012-12-28,Gestion - Accueil Telephonique,0,4,0,2096569015


In [286]:
X_sub = sub[['ass_hashed', 'slot', 'dayofweek']]

In [287]:
sub['prediction'] = rf.predict(X_sub)

In [288]:
sub.head()

,DATE,ASS_ASSIGNMENT,prediction,dayofweek,slot,ass_hashed
0,2012-12-28,CMS,0.123211,4,0,-1466160996
1,2012-12-28,Crises,1.562959,4,0,-272063095
2,2012-12-28,Domicile,0.260716,4,0,-1965427828
3,2012-12-28,Gestion,0.178383,4,0,822364246
4,2012-12-28,Gestion - Accueil Telephonique,0.178383,4,0,2096569015


In [289]:
sub.prediction.describe()

count    82909.000000
mean        10.205839
std         29.707240
min          0.002342
25%          0.260716
50%          0.724023
75%          4.163307
max        301.943487
Name: prediction, dtype: float64

In [290]:
real_sub = sub.iloc[:, :3]

In [291]:
real_sub.head()

,DATE,ASS_ASSIGNMENT,prediction
0,2012-12-28,CMS,0.123211
1,2012-12-28,Crises,1.562959
2,2012-12-28,Domicile,0.260716
3,2012-12-28,Gestion,0.178383
4,2012-12-28,Gestion - Accueil Telephonique,0.178383


In [292]:
real_sub.DATE = real_sub.DATE.astype(str) + ".000"

In [293]:
real_sub.to_csv('data/sub_chiaman_3.txt', sep='\t', index=None)